In [6]:
from elasticsearch import Elasticsearch

In [7]:
username = 'elastic'
password = '1234'
host = 'http://localhost:9200'  # e.g., 'localhost' or 'https://your-elasticsearch-domain.com'

# Create an Elasticsearch client instance
es = Elasticsearch(
    hosts=host,
    basic_auth=(username, password),
    verify_certs = False
)

# Check if the connection was successful
if es.ping():
    print('#'*30)
    print("Connected to Elasticsearch")
    print('#'*30)
else:
    print("Connection failed")

# es.info()

##############################
Connected to Elasticsearch
##############################


In [8]:
es.indices.get_alias(index='*')

ObjectApiResponse({'face_recognition': {'aliases': {}}, 'my_index1': {'aliases': {}}, 'saleem_testing1': {'aliases': {}}})

In [90]:
def convert_to_string(item):
    if isinstance(item, dict):
        for key, value in item.items():
            item[key] = convert_to_string(value)
    elif isinstance(item, list):
        for i in range(len(item)):
            item[i] = convert_to_string(item[i])
    else:
        try:
            # Attempt to convert to string
            item = str(item)+' '
        except:
            # Handle any exceptions during conversion
            item = None  # Or you can choose a different default value

    return item

def correct_values(item):
    if isinstance(item, dict):
        for key, value in item.items():
            item[key] = correct_values(value)
    elif isinstance(item, list):
        for i in range(len(item)):
            item[i] = correct_values(item[i])

    return item

documents = [
    {
        'name': 'xyz',
        'birth_encorporation_date': '[1996-01-01]',
        'data': {
            'summary': {
                'name': 'wxyz',
                'place_of_birth': 'khanpur',
                'dob': '1996',
                'age': 27,
                'range': [20, 27],
                'salary': [10000, 200000],
                'passport_number':'pass1234'
            }
        }
    },
    {
        'name': 'xyz',
        'birth_encorporation_date': '[1996-02]',
        'data': {
            'summary': {
                'name': 'abcd',
                'nationality': 'pakistan',
                'range': '[20-27]',
                'birth_year': '1996',
                'id_number':'id1234',
                
            }
        }
    },
    {
        'name': 'xyz',
        'birth_encorporation_date': '[1997-02]',
        'data': {
            'summary': {
                'name': 'abcd',
                'nationality': 'pakistan',
                'range': '[20-27]',
                'birth_year': 1997,
                'id_number':'id42',
            }
        }
    },
    {
        'name': 'abc',
        'birth_encorporation_date': '[1997-02]',
        'data': {
            'summary': {
                'name': 'abcd',
                'nationality': 'pakistan',
                'range': '[20-27]',
                'birth_year': 1997,
                'id_number':'id42',
            }
        }
    },
    {
        'name': 'uvs',
        'birth_encorporation_date': '[]',
        'data': {
            'summary': {
                'name': 'uvs',
                'nationality': 'pakistan',
                'date_of_birth': '1996-01-15',
                'siblings': {'brother': {'name': 'abc'}},
            }
        }
    }
]

# Correct and clean the values
for idx, doc in enumerate(documents):
    documents[idx] = correct_values(doc)

# Convert non-dict and non-list values to string
for idx, doc in enumerate(documents):
    documents[idx] = convert_to_string(doc)

display(documents)


[{'name': 'xyz ',
  'birth_encorporation_date': '[1996-01-01] ',
  'data': {'summary': {'name': 'wxyz ',
    'place_of_birth': 'khanpur ',
    'dob': '1996 ',
    'age': '27 ',
    'range': ['20 ', '27 '],
    'salary': ['10000 ', '200000 '],
    'passport_number': 'pass1234 '}}},
 {'name': 'xyz ',
  'birth_encorporation_date': '[1996-02] ',
  'data': {'summary': {'name': 'abcd ',
    'nationality': 'pakistan ',
    'range': '[20-27] ',
    'birth_year': '1996 ',
    'id_number': 'id1234 '}}},
 {'name': 'xyz ',
  'birth_encorporation_date': '[1997-02] ',
  'data': {'summary': {'name': 'abcd ',
    'nationality': 'pakistan ',
    'range': '[20-27] ',
    'birth_year': '1997 ',
    'id_number': 'id42 '}}},
 {'name': 'abc ',
  'birth_encorporation_date': '[1997-02] ',
  'data': {'summary': {'name': 'abcd ',
    'nationality': 'pakistan ',
    'range': '[20-27] ',
    'birth_year': '1997 ',
    'id_number': 'id42 '}}},
 {'name': 'uvs ',
  'birth_encorporation_date': '[] ',
  'data': {'summ

In [91]:

# Index name
index_name = 'my_index1'

# Delete index if it exists
try:
    es.indices.delete(index=index_name)
except:
    print('Index does not exist or could not be deleted')

# Mapping and settings
mapping = {
    'dynamic': True,  # Allow dynamic mapping for varying fields
    'properties': {
        'name': {'type': 'text'},
        'birth_encorporation_date': {'type': 'keyword'},
        'data': {
           'type': 'object'
            }
        }
}

settings = {
    'number_of_shards': 1,
    'number_of_replicas': 0
}

# Create index with mapping and settings
es.indices.create(index=index_name, body={'settings': settings, 'mappings': mapping})

# Index documents
for doc in documents:
    es.index(index=index_name, body=doc)



C:\Users\Desktop\AppData\Local\Temp\ipykernel_10440\478177309.py:28: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index=index_name, body={'settings': settings, 'mappings': mapping})
C:\Users\Desktop\AppData\Local\Temp\ipykernel_10440\478177309.py:32: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=index_name, body=doc)


In [92]:
q={
    "query": {
        "match_all": {}
    },
}
result = es.search(index=index_name, body=q)
print(es.count(index=index_name, body=q))
for i in result['hits']['hits']:
    display(i)
    

{'count': 1, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


C:\Users\Desktop\AppData\Local\Temp\ipykernel_10440\3814896417.py:6: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=q)
C:\Users\Desktop\AppData\Local\Temp\ipykernel_10440\3814896417.py:7: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  print(es.count(index=index_name, body=q))


{'_index': 'my_index1',
 '_id': 'oVX7MooBLCGmj3FM6q7Y',
 '_score': 1.0,
 '_source': {'name': 'xyz ',
  'birth_encorporation_date': '[1996-01-01] ',
  'data': {'summary': {'name': 'wxyz ',
    'place_of_birth': 'khanpur ',
    'dob': '1996 ',
    'age': '27 ',
    'range': ['20 ', '27 '],
    'salary': ['10000 ', '200000 '],
    'passport_number': 'pass1234 '}}}}

In [93]:
# Sample search query for nested field
def search_nested_records(key='', value=''):
    query = {
        'query': {
            
                'wildcard': {
                    key: f"*{value}*"
                }
                }
    }
    result = es.search(index=index_name, body=query)
    return result

# Example search for nested field
search_result = search_nested_records(key='data.summary.dob.keyword', value='1996')
for i in search_result['hits']['hits']:
    display(i)
    

C:\Users\Desktop\AppData\Local\Temp\ipykernel_10440\2490340115.py:11: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


{'_index': 'my_index1',
 '_id': 'oVX7MooBLCGmj3FM6q7Y',
 '_score': 1.0,
 '_source': {'name': 'xyz ',
  'birth_encorporation_date': '[1996-01-01] ',
  'data': {'summary': {'name': 'wxyz ',
    'place_of_birth': 'khanpur ',
    'dob': '1996 ',
    'age': '27 ',
    'range': ['20 ', '27 '],
    'salary': ['10000 ', '200000 '],
    'passport_number': 'pass1234 '}}}}

In [98]:
def search_nested_records(name='', birth_encorporation_date='',search_term=''):
    query = {
        "query": {
            "bool": {
                "must": [],
                "should": []
            }
        }
    }

    if name:
        query["query"]["bool"]["must"].append({"match": {"name": name}})

    if birth_encorporation_date:
        query["query"]["bool"]["should"].append(
            {
              'wildcard': {
                    'birth_encorporation_date': f"*{birth_encorporation_date}*",
                }  
            }
            )
        query["query"]["bool"]["should"].append(
            {
              'wildcard': {
                    'data.summary.birth_year.keyword': f"*{birth_encorporation_date}*",
                }  
            }
            )
        query["query"]["bool"]["should"].append(
            {
              'wildcard': {
                    'data.summary.dob.keyword': f"*{birth_encorporation_date}*"
                }  
            }
            )
    if search_term:
        query["query"]["bool"]["should"].append(
            {
               "query_string": {
                        "query": f"*{search_term}*",
                        "fields": ["*"],
                        "default_operator": "OR"
                    } 
            }
            )
    result = es.search(index=index_name, body=query)
    return result


# Example search for nested field
search_result = search_nested_records(name='', birth_encorporation_date='',search_term='id42')
for i in search_result['hits']['hits']:
    display(i)
    

C:\Users\Desktop\AppData\Local\Temp\ipykernel_10440\1885022914.py:46: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


{'_index': 'my_index1',
 '_id': 'o1X7MooBLCGmj3FM8K5s',
 '_score': 1.0,
 '_source': {'name': 'xyz ',
  'birth_encorporation_date': '[1997-02] ',
  'data': {'summary': {'name': 'abcd ',
    'nationality': 'pakistan ',
    'range': '[20-27] ',
    'birth_year': '1997 ',
    'id_number': 'id42 '}}}}

{'_index': 'my_index1',
 '_id': 'pFX7MooBLCGmj3FM8K7l',
 '_score': 1.0,
 '_source': {'name': 'abc ',
  'birth_encorporation_date': '[1997-02] ',
  'data': {'summary': {'name': 'abcd ',
    'nationality': 'pakistan ',
    'range': '[20-27] ',
    'birth_year': '1997 ',
    'id_number': 'id42 '}}}}